In [1]:
import pandas
import time
import argparse
import def_function
from def_class import *
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import numpy

In [2]:
__version__ = "V3.0(Editor) 2023-08-11"

In [3]:
# 前置参数-debug
ATSS_APA_cutoff = 3
log2FC_cutoff = 1.5
plot_x_maxnum = 5
check_counts = False
absolute_path = False
file_path = "F:/OneDrive/Master/Project/trans/data/"
input_sample_info_filename = "0000_sample_info.tsv"
input_df_filename = "0001_total_info.tsv"
input_pkl_filename = "0002_total_info.pkl"
output_length_counts_plot_filename = "0004_length_counts.png"
output_check_df_filename = "0004_check_df.tsv"
output_transcript_gene_plot_path = "0004_plot/"
output_relative_expression_filename = "0004_relative_expression.tsv"

In [ ]:
# 前置参数
parser = argparse.ArgumentParser()
parser.add_argument("--ATSS_APA_cutoff", dest="ATSS_APA_cutoff", required=False, type=int, default=3, help="=3,\t the cutoff value of isoform of ATSS-APA gene")
parser.add_argument("--log2FC_cutoff", dest="log2FC_cutoff", required=False, type=float, default=1.5, help="=1.5,\t the cutoff value of log2FC")
parser.add_argument("--plot_x_maxnum", dest="plot_x_maxnum", required=False, type=int, default=5, help="=5,\t the maxnum in xaxis")
parser.add_argument("--check_counts", dest="check_counts", required=False, action="store_true", help="output the plot about the relation of length and rep value of transcript")
parser.add_argument("--absolute_path", dest="absolute_path", required=False, action="store_true", help="use the absolute path")
parser.add_argument("--file_path", dest="file_path", required=False, type=str, default="./", help="=the data directory path")
parser.add_argument("--input_sample_info_filename", dest="input_sample_info_filename", required=False, type=str, default="0000_sample_info.tsv", help="=0000_sample_info.tsv,\t the metadata of samples")
parser.add_argument("--input_df_filename", dest="input_df_filename", required=False, type=str, default="0001_total_info.tsv", help="=0001_total_info.tsv,\t the output file of 0001.py")
parser.add_argument("--input_pkl_filename", dest="input_pkl_filename", required=False, type=str, default="0002_total_info.pkl", help="=0002_total_info.pkl,\t the output file of 0002.py")
parser.add_argument("--output_length_counts_plot_filename", dest="output_length_counts_plot_filename", required=False, type=str, default="0004_length_counts.png", help="=0004_length_counts.png,\t to confirm rep is not related to length of transcript")
parser.add_argument("--output_check_df_filename", dest="output_check_df_filename", required=False, type=str, default="0004_check_df.xlsx", help="=0004_check_df.xlsx,\t the output file for checking")
parser.add_argument("--output_transcript_gene_plot_path", dest="output_transcript_gene_plot_path", required=False, type=str, default="0004_plot/", help="=0004_plot/,\t the path of output plot")
parser.add_argument("--output_relative_expression_filename", dest="output_relative_expression_filename", required=False, type=str, default="0004_relative_expression.tsv", help="=0004_relative_expression.tsv,\t the output filename of cell_line expression")

args = parser.parse_args()
ATSS_APA_cutoff = args.ATSS_APA_cutoff
log2FC_cutoff = args.log2FC_cutoff
plot_x_maxnum = args.plot_x_maxnum
check_counts = args.check_counts
absolute_path = args.absolute_path
file_path = args.file_path
input_sample_info_filename = args.input_sample_info_filename
input_df_filename = args.input_df_filename
input_pkl_filename = args.input_pkl_filename
output_length_counts_plot_filename = args.output_length_counts_plot_filename
output_check_df_filename = args.output_check_df_filename
output_transcript_gene_plot_path = args.output_transcript_gene_plot_path
output_relative_expression_filename = args.output_relative_expression_filename

In [4]:
# 补充路径
if absolute_path is False:
    input_sample_info_filename = "{}{}".format(file_path, input_sample_info_filename)
    input_df_filename = "{}{}".format(file_path, input_df_filename)
    input_pkl_filename = "{}{}".format(file_path, input_pkl_filename)
    output_length_counts_plot_filename = "{}other/{}".format(file_path, output_length_counts_plot_filename)
    output_check_df_filename = "{}{}".format(file_path, output_check_df_filename)
    output_transcript_gene_plot_path = "{}{}".format(file_path, output_transcript_gene_plot_path)
    output_relative_expression_filename = "{}{}".format(file_path, output_relative_expression_filename)

In [ ]:
# 打印参数
print('\n')
print("[Date]{}".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))))
print("[Script]{}".format(__file__))
print("[Version]{}".format(__version__))
print("[Version]def_function: {}".format(def_function.__version__))
print("[Version]def_class: {}".format(class_version))
print("[Parament]ATSS_APA_cutoff: {}".format(ATSS_APA_cutoff))
print("[Parament]log2FC_cutoff: {}".format(log2FC_cutoff))
print("[Parament]plot_x_maxnum: {}".format(plot_x_maxnum))
print("[Parament]check_counts: {}".format(check_counts))
print("[Parament]file_path: {}".format(file_path))
print("[Parament]input_sample_info_filename: {}".format(input_sample_info_filename))
print("[Parament]input_df_filename: {}".format(input_df_filename))
print("[Parament]input_pkl_filename: {}".format(input_pkl_filename))
print("[Parament]output_length_counts_plot_filename: {}".format(output_length_counts_plot_filename))
print("[Parament]output_check_df_filename: {}".format(output_check_df_filename))
print("[Parament]output_transcript_gene_plot_path: {}".format(output_transcript_gene_plot_path))
print("[Parament]output_relative_expression_filename: {}".format(output_relative_expression_filename))
print('\n')

In [6]:
# 隐藏函数
def getSignificantTranscriptId(total, cellLineExpression, cellLine1, cellLine2, log2Cutoff=1.5, tab_level=0):
    '''
    change:
        先筛选出在cellLine1或cellLine2中存在表达的transcript
        然后再继续筛选出于cellLine1或cellLine2中特异性表达的transcript
    '''
    total = total
    cellLineExpression = cellLineExpression
    cellLine1 = cellLine1
    cellLine2 = cellLine2
    log2Cutoff = log2Cutoff

    # 获取在cellLine1及cellLine2中存在表达且富集于其中一种cellLine中的transcriptId
    # 只保留在cellLine1及cellLine2中存在表达的transcript
    df = cellLineExpression
    df[cellLine1] = df[cellLine1].map(lambda x: None if x==0 else x)
    df[cellLine2] = df[cellLine2].map(lambda x: None if x==0 else x)
    df = df.dropna(axis=0, subset=[cellLine1, cellLine2], how="all")
    # 根据log2FC筛选transcript
    df["log2FC_{}@{}".format(cellLine1, cellLine2)] = df["log2FC_{}@{}".format(cellLine1, cellLine2)].map(lambda x: x if abs(x)>=log2Cutoff or x==0 else None)
    df = df.dropna(axis=0, subset=["log2FC_{}@{}".format(cellLine1, cellLine2)], how="any")

    return df.index.to_list()


def countsGeneNum(df, ATSS_APA_gene, transcriptIdList, geneBiotype="all", transcriptBiotype="all", plot_x_maxnum=5):
    '''
    input:
        df, pandas.DataFrame, 具有列["gene_id"]且index.name为"transcriptId"
        ATSS_APA_gene, list, 存储了ATSS-APA型gene的id
        transcriptIdList, list, 存储了transcriptId, 这些transcript在指定cellLine中存在特异性表达
        plot_x_maxnum, int, 最大值, 高于此值的值将被修改为此值
    change:
        记录df中符合条件的gene的数量
            gene具有指定数量的特异性表达transcript
    output:
        isoformCounts, dict, 记录了gene的数量, key为gene所具有的特异性表达transcript数量, value为gene数量
                               额外含有key: geneBiotype, transcriptBiotype, cellLine1, cellLine2
    '''
    df = df
    ATSS_APA_gene = ATSS_APA_gene
    transcriptIdList = transcriptIdList
    geneBiotype = geneBiotype
    transcriptBiotype = transcriptBiotype
    plot_x_maxnum = plot_x_maxnum

    isoformCounts = {i: 0 for i in range(0, plot_x_maxnum+1)}

    # 保存geneBiotype及transcriptBiotype信息
    isoformCounts["geneBiotype"] = geneBiotype
    isoformCounts["transcriptBiotype"] = transcriptBiotype
    # 准备记录cellLine信息
    isoformCounts["cellLine1"] = None
    isoformCounts["cellLine2"] = None

    # 筛选ATSS_APA型gene
    df = df.loc[df["gene_id"].isin(ATSS_APA_gene), :]
    # 根据geneBiotype进行筛选
    df = df if geneBiotype == "all" else df.query("gene_biotype==@geneBiotype")
    # 根据transcriptBiotype进行筛选
    df = df if transcriptBiotype == "all" else df.query("transcript_biotype==@transcriptBiotype")
    # 记录符合geneBiotype及transcriptBiotype的gene的总数
    geneTotalNum = len(set(df["gene_id"]))

    # 筛选特异性表达的transcript
    df = df.loc[df.index.isin(transcriptIdList), :]
    # 记录不存在特异性表达transcript的ATSS-APA型gene的数量
    isoformCounts[0] = geneTotalNum - len(set(df["gene_id"]))

    df_group = df.groupby("gene_id")
    # 遍历每一个gene, 记录gene中具有特异性表达的transcript的种类数
    for geneId,transcriptId in df_group.groups.items():
        if len(transcriptId) > plot_x_maxnum:
            isoformCounts[plot_x_maxnum] = isoformCounts[plot_x_maxnum] + 1
        else:
            isoformCounts[len(transcriptId)] = isoformCounts[len(transcriptId)] + 1
    

    return isoformCounts

In [17]:
# 功能函数
def compareCellLineExpression(total, tab_level=0):
    transcriptCellLineExpression = {}
    # 遍历每一个transcript, 获取transcript在每一个cellLine中的平均相对表达量
    for geneId, geneObject in total.gene_dict.items():
        for transcriptId, transcriptObject in geneObject.transcript_dict.items():
            transcriptCellLineExpression[transcriptId] = {cellLine: transcriptObject["cellLineExpression"].get(cellLine, 0) for cellLine in total.cellLineInfo.keys()}
            transcriptCellLineExpression[transcriptId].update({"gene_id": geneObject.gene_id,
                                                               "gene_biotype": geneObject.gene_biotype,
                                                               "transcript_biotype": transcriptObject["transcript_biotype"]})
    # 准备表格的列名
    columns = ["gene_id", "gene_biotype", "transcript_biotype"] + list(total.cellLineInfo.keys())
    # 将transcript的相关信息转化为df
    transcriptCellLineExpression = pandas.DataFrame.from_dict(transcriptCellLineExpression,
                                                              orient="index", columns=columns)
    # 美化表格
    transcriptCellLineExpression.index.name = "transcript_id"

    cellLineList = list(total.cellLineInfo.keys())
    for cellLine1Index in range(0, len(cellLineList)-1):
        for cellLine2Index in range(cellLine1Index+1, len(cellLineList)):
            cellLine1 = cellLineList[cellLine1Index]
            cellLine2 = cellLineList[cellLine2Index]
            transcriptCellLineExpression["log2FC_{}@{}".format(cellLine1, cellLine2)] = transcriptCellLineExpression[cellLine1] / \
                                                                                        transcriptCellLineExpression[cellLine2]
            transcriptCellLineExpression["log2FC_{}@{}".format(cellLine1, cellLine2)] = transcriptCellLineExpression["log2FC_{}@{}".format(cellLine1, cellLine2)].map(lambda x: 0 if numpy.isinf(x) or x==0 else math.log(x, 2))
    
    return transcriptCellLineExpression


def getATSSAPAGeneId(total, ATSS_APA_cutoff=3):
    '''
    change:
        获取ATSS-APA型gene
    '''
    total = total
    ATSS_APA_cutoff = ATSS_APA_cutoff

    geneList = []  # 存储ATSS-APA型gene的geneId
    for geneId, geneObject in total.gene_dict.items():
        if geneObject.gene_classified == "ATSS-APA":
            # 确定该gene属于ATSS-APA型gene
            if len(geneObject.transcript_dict) >= ATSS_APA_cutoff:
                # 确定该gene的isoforms数量>=定值
                geneList.append(geneId)

    return geneList


def dfCountsGeneNum(total, cellLineExpression, log2FC_cutoff, geneBiotype, transcriptBiotype, ATSS_APA_gene, plot_x_maxnum, tab_level=0):
    '''
    input:
    change:
        获取所有的cellLine组合, 并统计具有不同isoforms的gene的数量
    output:
        df, pandas.DataFrame, 含有列["cellLine1", "cellLine2", "geneBiotype", "transcriptBiotype", 0, 1, 2, ...]
    '''
    total = total
    cellLineExpression = cellLineExpression
    log2FC_cutoff = log2FC_cutoff
    geneBiotype = geneBiotype
    transcriptBiotype = transcriptBiotype
    ATSS_APA_gene = ATSS_APA_gene
    plot_x_maxnum = plot_x_maxnum

    num = 0
    isoformCounts = {}
    # 遍历所有的cellLine组合, 统计其gene数量
    for cellLine1, cellLine2 in cellLineCombination(cl=list(total.cellLineInfo.keys())):
        num += 1
        transcriptIdList = getSignificantTranscriptId(total=total, cellLineExpression=cellLineExpression,
                                                cellLine1=cellLine1, cellLine2=cellLine2,
                                                log2Cutoff=log2FC_cutoff)
        isoformCountsTemp = countsGeneNum(df=cellLineExpression, geneBiotype=geneBiotype, transcriptBiotype=transcriptBiotype,
                                ATSS_APA_gene=ATSS_APA_gene, transcriptIdList=transcriptIdList,
                                plot_x_maxnum=plot_x_maxnum)
        isoformCountsTemp["cellLine1"] = cellLine1
        isoformCountsTemp["cellLine2"] = cellLine2
        isoformCounts[num] = isoformCountsTemp
    # 生成df
    columns = ["cellLine1", "cellLine2", "geneBiotype", "transcriptBiotype"]
    columns = columns + [i for i in range(0, plot_x_maxnum+1)]
    df = pandas.DataFrame.from_dict(isoformCounts, orient="index",
                            columns=columns)
    
    return df




In [5]:
# 绘图函数
@def_function.log
def output_length_counts_check_plot(sample_info, total_info, output_filename, tab_level=0):
    """
    input:
        sample_info, df
        total_info, df
        output_filename, str
    change:
        get the plot which xaxis is length of transcript and yaxis is counts of transcript
    output:
        None
    """
    sample_info = sample_info
    total_info = total_info
    output_filename = output_filename

    fig_info_dict = {}
    for sample in sample_info["GEO_accession"]:
        temp_df = total_info[[sample, "transcript_start", "transcript_end"]]
        temp_df = temp_df.loc[temp_df[sample]>0, :]  # get the rows of count>0
        temp_df["length"] = temp_df["transcript_end"] - temp_df["transcript_start"]
        temp_df = temp_df[[sample, "length"]]

        fig_info = go.Scatter(x=temp_df["length"],
                              y=temp_df[sample],
                              name=sample,
                              mode="markers",
                              marker={"size": 2.5})
        
        fig_info_dict[sample] = fig_info

    #rows = math.ceil(math.sqrt(sample_info.shape[0]))
    #cols = math.ceil(sample_info.shape[0]/rows)
    fig = make_subplots(rows=1, cols=1,
                        shared_xaxes=True, shared_yaxes=True)
    for sample, fig_info in fig_info_dict.items():
        fig.add_trace(fig_info)
    
    layout = {"title": {"font": {"family": "Arial",
                                 "size": 12,
                                 "color": "black"},
                        },
              "font": {"family": "Arial",
                       "size": 10,
                       "color": "black"},
              "margin": {'l':20, 'r':20, 't':20, 'b':20},
              "xaxis": {"title": {"text": "transcripts length",
                                  "font": {"family": "Arial",
                                           "color": "black",
                                           "size": 12},
                                  "standoff": 0.5},
                        "showline": True,
                        "linecolor": "black",
                        },
              "yaxis": {"title": {"text": "transcripts counts",
                                  "font": {"family": "Arial",
                                           "color": "black",
                                           "size": 12},
                                  "standoff": 0.5},
                        "showline": True,
                        "linecolor": "black",
                        },
              "paper_bgcolor": "white",
              "plot_bgcolor": "white",
              "showlegend": False
              }

    fig.update_layout(layout)

    fig.write_image(output_length_counts_plot_filename, scale=4)

    return None


def make_transcript_gene_plot(df):
    '''
    input:
        df, pandas.DataFrame, 含有列['x', 'y']
    '''
    df = df

    plot = go.Figure(go.Bar(x=df['x'], y=df['y'],
                            width=0.9,
                            text=[int(df.query('x==0')['y'])],
                            textposition="outside",
                            textfont={"family": "Arial", "color": "black", "size": 12},
                            textangle=-90,
                            marker={"color": "purple",
                                    "line": {"color": "black",
                                            "width": 0.5},
                                    }))
    layout = {"width": 250, "height": 400,
              "title": {"text": "{}/{}".format(df['y'].sum()-int(df.query('x==0')['y']), df['y'].sum()),
                        "font": {"family": "Arial",
                                 "color": "black",
                                 "size": 12},
                        'x': 0.8,},
              "margin": {'l':15, 'r':15, 't':25, 'b':15}, 
              "barmode": "stack",
              "font": {"family": "Arial",
                       "color": "black",
                       "size": 10},
              "xaxis": {"title": {"text": "Per gene, 5'-3' isoforms <br> with a significant 5'-3' link",
                                  "font": {"family": "Arial",
                                           "color": "black",
                                           "size": 12},
                                  "standoff": 0,},
                        "linecolor": "black",
                        "dtick": 1,
                        "ticksuffix": '+',
                        "showticksuffix": "last",
                        },
              "yaxis": {"title": {"text": "Number of Genes",
                                  "font": {"family": "Arial",
                                           "color": "black",
                                           "size": 12},
                                  "standoff": 0,},
                        "showgrid": False,
                        "showline": True,
                        "linecolor": "black",
                        "tickfont": {"size": 10}
                        },
              "plot_bgcolor": "white",
              "bargap": 0, "bargroupgap": 0,}
    plot = plot.update_layout(layout)

    return plot


'\n@def_function.log\ndef output_transcript_gene_plot(result_counts, file_path, tab_level=0):\n    result_counts = result_counts\n    file_path = output_transcript_gene_plot_path\n\n    plot_info_df = pandas.DataFrame()\n    for cell_line_compare, plot_info_dict in result_counts.items():\n        for gene_transcript, value_dict in plot_info_dict.items():\n            [cell_line_1,cell_line_2] = cell_line_compare.split(\'@\')\n            [gene_biotype,transcript_biotype] = gene_transcript.split(\'/\')\n            temp_df = pandas.DataFrame([[cell_line_1, cell_line_2, gene_biotype, transcript_biotype]+[-1 for i in range(0,plot_x_maxnum+1)]],\n                                        columns=["cell_line_1", "cell_line_2", "gene_biotype", "transcript_biotype"]+list(range(0, plot_x_maxnum+1)))\n            for num, counts in value_dict.items():\n                temp_df[num] = counts\n            plot_info_df = pandas.concat([plot_info_df, temp_df], axis=0, ignore_index=True)\n\n    for i i

In [13]:
# 生成器
def cellLineCombination(cl):
    '''
    input:
        cl, list, [<cellLine1>, <cellLine2>, <cellLine3>, ...]
    generator:
        以[cellLine1, cellLine2]形式返回给定list中的每一对组合
    '''
    cl = cl
    for cellLine1Index in range(0, len(cl)-1):
        for cellLine2Index in range(cellLine1Index+1, len(cl)):
            # 遍历了每一对cellLine组合
            cellLine1 = cl[cellLine1Index]
            cellLine2 = cl[cellLine2Index]
            yield [cellLine1, cellLine2]

In [21]:
if __name__ == "__main__":
    # load data
    sample_info = def_function.load_sample_info(filename=input_sample_info_filename, tab_level=0)
    total_info = pandas.read_csv(input_df_filename, sep='\t', index_col=0)
    with open(input_pkl_filename, "rb") as file:
        total = pickle.load(file)
    
    # output check plot to confirm the relationship of length and counts
    if check_counts is True:
        output_length_counts_check_plot(sample_info=sample_info,
                                        total_info=total_info,
                                        output_filename=output_length_counts_plot_filename,
                                        tab_level=0)

    # compare expression of two cell line
    # and filter transcripts according to log2FC
    cellLineExpression = compareCellLineExpression(total=total)
    ATSS_APA_gene = getATSSAPAGeneId(total=total, ATSS_APA_cutoff=ATSS_APA_cutoff)
    # 将geneBiotype及transcriptBiotype归类为三种类型: ["protein_coding", "nonprotein_coding", "un_classified"]
    df = cellLineExpression.copy()
    df["gene_biotype"] = df["gene_biotype"].map(lambda x: "nonprotein_coding" if x not in ["protein_coding", "un_classified"] else x)
    df["transcript_biotype"] = df["transcript_biotype"].map(lambda x: "nonprotein_coding" if x not in ["protein_coding", "un_classified"] else x)
    # 获取每种geneBiotype及transcriptBiotype的gene的数目, 存储于dfCounts
    dfCounts = pandas.DataFrame()
    for geneBiotype in ["all", "protein_coding", "nonprotein_coding", "un_classified"]:
        for transcriptBiotype in ["all", "protein_coding", "nonprotein_coding", "un_classified"]:
            dfTemp = dfCountsGeneNum(total=total, cellLineExpression=df,
                                     log2FC_cutoff=log2FC_cutoff, plot_x_maxnum=plot_x_maxnum,
                                     geneBiotype=geneBiotype, transcriptBiotype=transcriptBiotype,
                                     ATSS_APA_gene=ATSS_APA_gene, tab_level=0)
            dfCounts = pandas.concat([dfCounts, dfTemp], axis=0, ignore_index=True)

    # 绘图
    plotDict = dfCounts.to_dict("index")
    for plotId, plotInfo in plotDict.items():
        dfTemp = pandas.DataFrame({'x': [i for i in range(0, plot_x_maxnum+1)],
                                'y': [plotInfo[i] for i in range(0, plot_x_maxnum+1)]})
        plotDict[plotId].update({"plot": make_transcript_gene_plot(dfTemp)})

    # 输出结果
    columns = ["cellLine1", "cellLine2", "geneBiotype", "transcriptBiotype"]
    columns = columns + [i for i in range(0, plot_x_maxnum+1)]
    df = pandas.DataFrame.from_dict(plotDict, orient="index",
                                    columns=columns)
    df.to_csv(output_check_df_filename, sep='\t')
    cellLineExpression.to_csv(output_relative_expression_filename, sep='\t')
    for plotId, plotInfo in plotDict.items():
        plotInfo["plot"].write_image("{}{}_{}_{}.svg".format(output_transcript_gene_plot_path, plotId,
                                                             plotInfo["cellLine1"], plotInfo["cellLine2"]))

    print("[{}]All blocks finished.".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))))



[Function]load_sample_info start.
	[Time]2023-08-11 15:41:26
	[Paraments]filename: F:/OneDrive/Master/Project/trans/data/0000_sample_info.tsv
	[Paraments]tab_level: 0
[2023-08-11 15:41:26]load_sample_info finished.
[2023-08-11 15:41:36]All blocks finished.


---

In [ ]:
"""
1.对每一个样本中的transcript的表达量进行标准化
  2.1不确定原始数据的rep是如何得到的, 不知道应该用什么计算方法
  2.2姑且认为rep值已经对基因长度进行了校正, 因为rep值将近80%都<=10, 值很低
  2.3所以认为 一个transcript在一个样本中的相对表达量 = 该transcript的rep值 / 该样本所有transcript的rep值的和
  2.4再计算该transcript在相同疾病类型中的不同样本中的平均表达量= 该transcript在一种类型疾病中所有样本的相对表达量的加和 / 该类型疾病的样本数
  2.4再了解一下rep值与transcript长度是否相关
2.筛选得到在指定的两种细胞系类型的样本中都存在表达ATSS-APA基因
  2.1根据所有样本的数据筛选ATSS-APA基因id
  2.2获取每种细胞系中存在表达的基因id, 以list形式存储
  2.3分别筛选出在单一一种细胞系类型中表达的基因，然后将这些基因合并后去重
3.
"""

---

调试

In [142]:
cellLineList = list(total.cellLineInfo.keys())
for i in range(0, len(cellLineList)-1):
    for j in range(i+1, len(cellLineList)):
        cellLine1 = cellLineList[i]
        cellLine2 = cellLineList[j]
        print(cellLine1, cellLine2)

A673 Caco-2


调试

---